In [ ]:
pip install matplotlib

In [ ]:
import wave
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#from scipy.signal import find_peaks
from IPython.display import Audio

### MAIN

In [ ]:
# MÓDULO DE TODAS LAS FUNCIONES
def escuchar(i):
    display(Audio(dataset.loc[i,'Path'],autoplay = True))
    return

# Función para calcular el espectro de frecuencia usando FFT y fftshift
def calcular_fft(signal, rate):
    window = np.hanning(len(signal))
    signal_windowed = signal * window
    n = len(signal)
    freq = np.fft.fftshift(np.fft.fftfreq(n, d=1/rate))
    fft_values = np.fft.fftshift(np.abs(np.fft.fft(signal_windowed)))
    return freq, fft_values

# Función para cargar los datos de un archivo .wav
def cargar_audio(audio_path):
    with wave.open(audio_path, 'r') as wav_file:
        n_frames = wav_file.getnframes()
        n_channels = wav_file.getnchannels()
        rate = wav_file.getframerate()
        
        # Leer la señal del archivo
        signal = wav_file.readframes(n_frames)
        signal = np.frombuffer(signal, dtype=np.int16)
        
        if n_channels == 2:  # Convertir a mono si hay 2 canales
            signal = signal.reshape(-1, 2)
            signal = signal.mean(axis=1)
    return signal, rate


def pintar_tiempo(matriz_senales,i,len_signal,rate):
    time = np.linspace(0, len_signal / rate, len_signal)
    signal = matriz_senales[i,:len_signal]
    plt.figure(figsize=(10, 4))
    plt.plot(time, signal)
    plt.title(f'Dominio Temporal')#: Audio {i+1}')
    plt.xlabel('Tiempo (s)')
    plt.ylabel('Amplitud')
    plt.show()

def frecuencia_buscada(f,freq,rate):
    return int(np.size(freq) / 2 + f /( rate / np.size(freq)))

def encontrar_indice_maximo_local(freq,fft_values,frecuencia,rate, tol=50):
    np.argmax(fft_values[frecuencia_buscada(frecuencia,freq,rate)-tol:frecuencia_buscada(frecuencia,freq,rate)+tol])
    return frecuencia_buscada(frecuencia,freq,rate) - tol + np.argmax(fft_values[frecuencia_buscada(frecuencia,freq,rate)-tol:frecuencia_buscada(frecuencia,freq,rate)+tol])

def construir_vector_alpha_no_norm(freq,fft_values,ticks_frecuencia,rate):
    lista = np.zeros(len(ticks_frecuencia))
    for i in np.arange(len(ticks_frecuencia)):
        lista[i]=(20 * np.log10(fft_values[encontrar_indice_maximo_local(freq,fft_values,ticks_frecuencia[i],rate)]))
        lista[i]=((fft_values[encontrar_indice_maximo_local(freq,fft_values,ticks_frecuencia[i],rate)]))
    return lista

def alpha_a_voltios_rms(alpha):
    for i in np.arange(len(alpha)):
        alpha[i:] = 10**(alpha[i]/20)

def construir_vector_alpha(alpha_no_norm):
    cuadrado = [i**2 for i in alpha_no_norm]
    vector = alpha_no_norm/np.sqrt(np.sum(cuadrado))
    return vector

def pintar_freq(freq,ticks_frecuencia,j,fft_values):    
    margen = ticks_frecuencia[-1]
    plt.figure(figsize=(8, 4))
    plt.plot(freq,20 * np.log10(fft_values))
    for i in np.arange(len(ticks_frecuencia)):
        m = int(encontrar_indice_maximo_local(freq,fft_values,ticks_frecuencia[i],22050))
        #print(m)
        
        plt.plot([freq[m]],20 * np.log10(fft_values[encontrar_indice_maximo_local(freq,fft_values,ticks_frecuencia[i],22050)]),'ro')
        plt.plot([ticks_frecuencia[i]],20 * np.log10(fft_values[frecuencia_buscada(ticks_frecuencia[i],freq,22050)]),'yo')
        #plt.plot([ticks_frecuencia[i]],20 * np.log10(fft_values[encontrar_indice_maximo_local(freq,fft_values,ticks_frecuencia[i],22050)]),'ro')
    plt.xticks(ticks_frecuencia)

    plt.title(f"Espectro de frecuencia del audio {j}")
    plt.xlabel("Frecuencia (Hz)")
    plt.ylabel("Magnitud")
    plt.xlim(0, margen + 50)  # Opcional: rate/2 : mostrar solo la mitad positiva
    plt.grid(True)
    plt.show()

def pintar_frec(freq,i,fft_values):
    plt.figure(figsize=(10, 4))
    plt.plot(freq[int(len(freq)/2):], 20 * np.log10(fft_values[int(len(freq)/2):]))  # Amplitud en dB
    plt.title(f'Dominio de Frecuencia : Audio {i+1}')
    plt.xlabel('Frecuencia (Hz)')
    plt.ylabel('Amplitud (dB)')
    plt.show()

def pintar_frec_norm(freq,i,amplitudes_db,features):
    plt.figure(figsize=(10, 4))
    plt.plot(freq, amplitudes_db)  # Amplitud en dB
    plt.title(f'Dominio de Frecuencia (Normalizado): Audio {i}')
    plt.xlim(0,max(ticks_frecuencia)+150)
    plt.xticks(ticks_frecuencia)
    plt.yticks(features)
    plt.xlabel('Frecuencia (Hz)')
    plt.ylabel('Amplitud (dB)')
    plt.grid()
    plt.show()

def indice_frecuencia(frecuencia,amplitudes_db,rate):
    #
    # Devuelve el índice en el array freq en el que se encuentra la frecuencia para cada señal
    #
    return int(len(amplitudes_db)*frecuencia/rate*2)

def evaluar_maximo_interv(f_obj,amplitudes_db,tol):
    pico=np.argmax(amplitudes_db[f_obj-tol:f_obj+tol])
    #corrección del pico teoria-practica
    pico = f_obj + pico -tol-1
    return amplitudes_db[pico]

def etiquetar(data):
    y = np.array(data['Instrument (in full)']) # Etiquetas
    y = np.zeros(len(data),dtype=int)
    # Poner las etiquetas
    for i in np.arange(len(data)): 
        if np.array(data['Instrument (in full)'][i])=='Saxofon_Alto':
            y[i] = 0
        elif np.array(data['Instrument (in full)'][i])=='Clarinete':
            y[i] = 1
        elif np.array(data['Instrument (in full)'][i])=='Piano':
            y[i] = 2
        else :
            y[i] = 3
    return y

# Función Softmax
def softmax(z):
    exp_z = np.exp(z) 
    return exp_z / np.sum(exp_z, axis=0)

# Función Sigmoide
def sigmoid(z):
    return 1 / (1 + np.exp(-z))
def to_one_hot(y, num_classes):
        one_hot = np.zeros((len(y), num_classes))
        for i, label in enumerate(y):
            one_hot[i, int(label)] = 1
        return one_hot



In [ ]:
dataset = 'base_de_datos.csv'
data = pd.read_csv(dataset)

# Variables globales
num_armonicos = 4
tol = 50

ticks_frecuencia = []
for i in range(num_armonicos):
    ticks_frecuencia.append(221*(2*(i+1)))

# Matriz para almacenar todas las señales
n_audios = len(data)
longitud_maxima = 0  # Para determinar la longitud máxima de las señales


# Función para cargar los datos de un archivo .wav
def cargar_audio(audio_path):
    with wave.open(audio_path, 'r') as wav_file:
        n_frames = wav_file.getnframes()
        n_channels = wav_file.getnchannels()
        rate = wav_file.getframerate()
        
        # Leer la señal del archivo
        signal = wav_file.readframes(n_frames)
        signal = np.frombuffer(signal, dtype=np.int16)
        
        if n_channels == 2:  # Convertir a mono si hay 2 canales
            signal = signal.reshape(-1, 2)
            signal = signal.mean(axis=1)
    return signal, rate


# Primera pasada para calcular la longitud máxima
for audio_path in data['Path']:
    signal, _ = cargar_audio(audio_path)
    longitud_maxima = max(longitud_maxima, len(signal))

# Crear una matriz para almacenar las señales con padding
matriz_senales = np.zeros((n_audios, longitud_maxima))

alpha = np.zeros((len(data),len(ticks_frecuencia)))
# Segunda pasada para cargar las señales y almacenarlas en la matriz
for i, audio_path in enumerate(data['Path'][:]):
    signal, rate = cargar_audio(audio_path)
    matriz_senales[i, :len(signal)] = signal

    # Calcular el espectro en frecuencia
    freq, fft_values = calcular_fft(matriz_senales[i], rate)
     # Normalizar la señal
    fft_values = fft_values/fft_values.sum()

    alpha_no_norm = np.zeros(len(ticks_frecuencia))
    alpha_no_norm = construir_vector_alpha_no_norm(freq,fft_values,ticks_frecuencia,rate)
    alpha[i] = construir_vector_alpha(alpha_no_norm)
    assert np.abs(np.sum(alpha[i]**2)-1)<1e6

y = etiquetar(data)

### Pintar esfera

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

# Verify that each point lies on the unit sphere
for point in alpha:
    x, c, z = point
    assert np.isclose(x**2 + c**2 + z**2, 1), f"Point {point} does not lie on the unit sphere"

# Extract x, y, z coordinates from data points
x_coords = [point[0] for point in alpha]
y_coords = [point[1] for point in alpha]
z_coords = [point[2] for point in alpha]

# Colores para cada clase (puedes personalizarlos)
colores = ['b', 'orange', 'g', 'm']  # rojo, verde, azul, magenta
#colores = {0: 'r', 1: 'g', 2: 'b', 3: 'orange'}


# Asigna color a cada punto según su etiqueta
colores_por_punto = [colores[int(label)] for label in y]

# Crea la figura 3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

#for clase in np.unique(y):
#    indices = np.where(y == clase)
#    ax.scatter(
#        x_coords[indices], y_coords[indices], z_coords[indices],
#        c=colores[clase], label=etiquetas[clase], s=40
#    )

# Dibuja los puntos con su color correspondiente
ax.scatter(x_coords, y_coords, z_coords, c=colores_por_punto, marker='o')


# Plot the unit sphere for reference
u = np.linspace(0, np.pi/2, 100)
v = np.linspace(0, np.pi/2 , 100)
x_sphere = np.outer(np.cos(u), np.sin(v))
y_sphere = np.outer(np.sin(u), np.sin(v))
z_sphere = np.outer(np.ones(np.size(u)), np.cos(v))
ax.plot_surface(x_sphere, y_sphere, z_sphere, color='blue', alpha=0.1)

punto_destacado = [0.77962569, 0.40845272, 0.47471061]
#ax.scatter([punto_destacado[0]], [punto_destacado[1]], [punto_destacado[2]], color='r', s=100, label='Punto destacado')  # s=100 para hacerlo más grande

# Set labels
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')


ax.view_init(elev=30, azim=45)  # Puedes ajustar estos valores

from matplotlib.patches import Patch

# Supongamos que usaste estos colores para las clases
colores = {0: 'b', 1: 'orange', 2: 'g', 3: 'm'}
#etiquetas = {0: 'Piano', 1: 'Saxofón ', 2: 'Clarinete'}#, 3: 'Otro'}
etiquetas = {0: 'Saxofón', 1: 'Clarinete', 2: 'Piano'}#, 3: 'Otro'}

# Crear lista de "leyenda manual"
leyenda_manual = [Patch(color=colores[i], label=etiquetas[i]) for i in etiquetas]

# Añadir leyenda manual
ax.legend(handles=leyenda_manual, loc='upper right')

ax.set_title("Representación 3 primeros armónicos")
# Show plot
plt.show()


## Regresion



In [ ]:
import numpy as np

# Función sigmoide
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Función de pérdida logarítmica
def log_loss(y_true, y_pred):
    
    return -np.sum(y_true * np.log(y_pred + 1e-15) + (1 - y_true) * np.log(1 - y_pred + 1e-15))/len(y_true)

#### Comparación saxo clarinete

In [ ]:
# Preparar los datos (X = alpha, y = etiquetas binarias: 1 si saxofón, 0 si no)
X = np.array(alpha[:136])
y_all = np.array([1 if label == 0 else 0 for label in y[:136]])

indices = np.arange(68)
np.random.shuffle(indices)  # Mezcla los índices
print(indices)
X[:68] = X[indices]
y_all[:68] = y_all[indices]
X[68:] = X[68+indices]
y_all[68:] = y_all[68+indices]
# 80% entrenamiento, 20% test
split_index = int(0.75 * len(X))
indice_train = 55
split_index = np.concatenate((np.arange(0,indice_train),np.arange(68,68+indice_train)))
index_test = np.concatenate((np.arange(indice_train,68),np.arange(68+indice_train,68*2)))

X_train = X[split_index]
y_train = y_all[split_index]
X_test = X[index_test]
y_test = y_all[index_test]
#
#
indices = np.arange(len(X_train))
np.random.shuffle(indices)  # Mezcla los índices
#print(indices)
# Aplicamos la mezcla a ambos
X_train = X_train[indices]
y_train = y_train[indices]
#
#
indices = np.arange(len(X_test))
np.random.shuffle(indices)  # Mezcla los índices
#print(indices)
#
## Aplicamos la mezcla a ambos
X_test = X_test[indices]
y_test = y_test[indices]
#
# Inicializar pesos y sesgo
weights = np.zeros(X.shape[1])
bias = 0

# Parámetros del modelo
learning_rate = 0.75
epochs = 100000
loss = np.zeros(epochs)

# Entrenamiento
for epoch in range(epochs):
    linear_model = np.dot(X_train, weights) + bias      # z = W*x+b
    y_pred = sigmoid(linear_model)                      # a = g(z)
    
    # Gradientes
    dw = np.dot(X_train.T, (y_pred - y_train)) / y_train.size
    db = np.sum(y_pred - y_train) / y_train.size
    
    # Actualización
    weights -= learning_rate * dw
    bias -= learning_rate * db
    #print(f"y train {epoch}",y_train)
    #print(f"y pred{epoch}",y_pred)
    # Mostrar pérdida
    loss[epoch] = log_loss(y_train,y_pred)
    if epoch - epochs+1 == 0:
        lossy = log_loss(y_train, y_pred)
        print(f'Época {epoch}, Pérdida: {lossy:.4f}')


plt.figure(figsize=(8, 4))
eje = np.arange(epochs)
plt.plot(eje,loss)
plt.title(f"Ee")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Magnitud")
plt.grid(True)
plt.show()

def error(loss,i):
    return loss[i-1]-loss[i]

#plt.figure(figsize=(8, 4))
#eje = np.arange(epochs)
#plt.plot(eje[2:],(error(loss,eje[2:])))
##plt.plot(eje[2:],20*np.log10(error(loss,eje[2:])))
#plt.yscale('log')
#plt.title(f"Diferencias entre epocas en log-loss")
#plt.xlabel("Frecuencia (Hz)")
#plt.ylabel("Magnitud")
#plt.grid(True)
#plt.show()

# Evaluación en test
y_pred_test = sigmoid(np.dot(X_test, weights) + bias)
y_pred_class_test = [1 if i > 0.5 else 0 for i in y_pred_test]

accuracy = np.mean(y_pred_class_test == y_test)
print(f'Precisión del clasificador en test: {accuracy * 100:.2f}%')
from collections import Counter
print("Predicciones en test:", Counter(y_pred_class_test))
print("Etiquetas reales:", Counter(y_test))



In [ ]:
# Crear una cuadrícula para visualizar los vectores
grid = np.zeros((2, len(y_pred_class_test)))

# Asignar los valores a la cuadrícula
grid[0, :] = y_pred_class_test
grid[1, :] = y_test

# Crear un mapa de colores
cmap = plt.cm.get_cmap('coolwarm', 2)

# Graficar la cuadrícula
plt.imshow(grid, cmap=cmap, aspect='auto')

# Etiquetas y título
plt.title('Comparación de y_pred_class y y')
plt.xlabel('Índice')
plt.ylabel('Vectores')
plt.yticks([0, 1], ['y_pred_class', 'y'])

# Barra de color
cbar = plt.colorbar(ticks=[0, 1])
cbar.ax.set_yticklabels(['0', '1'])

# Mostrar la gráfica
plt.show()

### Comparación saxo piano

In [ ]:
# Preparar los datos (X = alpha, y = etiquetas binarias: 1 si saxofón, 0 si no)
X = np.concatenate((alpha[:68],alpha[68*2:]))
y_all = np.array([1 if label == 0 else 0 for label in np.concatenate((y[:68], y[68*2:]))])

indices = np.arange(68)
np.random.shuffle(indices)  # Mezcla los índices
print(indices)
X[:68] = X[indices]
y_all[:68] = y_all[indices]
X[68:] = X[68+indices]
y_all[68:] = y_all[68+indices]
# 80% entrenamiento, 20% test
split_index = int(0.75 * len(X))
indice_train = 55
split_index = np.concatenate((np.arange(0,indice_train),np.arange(68,68+indice_train)))
index_test = np.concatenate((np.arange(indice_train,68),np.arange(68+indice_train,68*2)))

X_train = X[split_index]
y_train = y_all[split_index]
X_test = X[index_test]
y_test = y_all[index_test]
#
#
indices = np.arange(len(X_train))
np.random.shuffle(indices)  # Mezcla los índices
#print(indices)
# Aplicamos la mezcla a ambos
X_train = X_train[indices]
y_train = y_train[indices]
#
#
indices = np.arange(len(X_test))
np.random.shuffle(indices)  # Mezcla los índices
#print(indices)
#
## Aplicamos la mezcla a ambos
X_test = X_test[indices]
y_test = y_test[indices]
#
# Inicializar pesos y sesgo
weights = np.zeros(X.shape[1])
bias = 0

# Parámetros del modelo
learning_rate = 0.75
epochs = 100000
loss = np.zeros(epochs)

# Entrenamiento
for epoch in range(epochs):
    linear_model = np.dot(X_train, weights) + bias      # z = W*x+b
    y_pred = sigmoid(linear_model)                      # a = g(z)
    
    # Gradientes
    dw = np.dot(X_train.T, (y_pred - y_train)) / y_train.size
    db = np.sum(y_pred - y_train) / y_train.size
    
    # Actualización
    weights -= learning_rate * dw
    bias -= learning_rate * db
    #print(f"y train {epoch}",y_train)
    #print(f"y pred{epoch}",y_pred)
    # Mostrar pérdida
    loss[epoch] = log_loss(y_train,y_pred)
    if epoch - epochs+1 == 0:
        lossy = log_loss(y_train, y_pred)
        print(f'Época {epoch}, Pérdida: {lossy:.4f}')

def error(loss,i):
    return loss[i-1]-loss[i]

plt.figure(figsize=(8, 4))
eje = np.arange(epochs)
plt.plot(eje,loss)
plt.title(f"Ee")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Magnitud")
plt.grid(True)
plt.show()

# Evaluación en test
y_pred_test = sigmoid(np.dot(X_test, weights) + bias)
y_pred_class_test = [1 if i > 0.5 else 0 for i in y_pred_test]

accuracy = np.mean(y_pred_class_test == y_test)
print(f'Precisión del clasificador en test: {accuracy * 100:.2f}%')
from collections import Counter
print("Predicciones en test:", Counter(y_pred_class_test))
print("Etiquetas reales:", Counter(y_test))

In [ ]:
# Crear una cuadrícula para visualizar los vectores
grid = np.zeros((2, len(y_pred_class_test)))

# Asignar los valores a la cuadrícula
grid[0, :] = y_pred_class_test
grid[1, :] = y_test

# Crear un mapa de colores
cmap = plt.cm.get_cmap('coolwarm', 2)

# Graficar la cuadrícula
plt.imshow(grid, cmap=cmap, aspect='auto')

# Etiquetas y título
plt.title('Comparación de y_pred_class y y')
plt.xlabel('Índice')
plt.ylabel('Vectores')
plt.yticks([0, 1], ['y_pred_class', 'y'])

# Barra de color
cbar = plt.colorbar(ticks=[0, 1])
cbar.ax.set_yticklabels(['0', '1'])

# Mostrar la gráfica
plt.show()

### Comparación Clarinete y Piano

In [ ]:
# Preparar los datos (X = alpha, y = etiquetas binarias: 1 si saxofón, 0 si no)
X = np.concatenate((alpha[68:136],alpha[68*2:]))
y_all = np.array([1 if label == 1 else 0 for label in np.concatenate((y[68:136], y[68*2:]))])

indices = np.arange(68)
np.random.shuffle(indices)  # Mezcla los índices
X[:68] = X[indices]
y_all[:68] = y_all[indices]
X[68:] = X[68+indices]
y_all[68:] = y_all[68+indices]
# 80% entrenamiento, 20% test
split_index = int(0.75 * len(X))
indice_train = 55
split_index = np.concatenate((np.arange(0,indice_train),np.arange(68,68+indice_train)))
index_test = np.concatenate((np.arange(indice_train,68),np.arange(68+indice_train,68*2)))

X_train = X[split_index]
y_train = y_all[split_index]
X_test = X[index_test]
y_test = y_all[index_test]
#
#
indices = np.arange(len(X_train))
np.random.shuffle(indices)  # Mezcla los índices
#print(indices)
# Aplicamos la mezcla a ambos
X_train = X_train[indices]
y_train = y_train[indices]
#
#
indices = np.arange(len(X_test))
np.random.shuffle(indices)  # Mezcla los índices
#print(indices)
#
## Aplicamos la mezcla a ambos
X_test = X_test[indices]
y_test = y_test[indices]
#
# Inicializar pesos y sesgo
weights = np.zeros(X.shape[1])
bias = 0

# Parámetros del modelo
learning_rate = 0.75
epochs = 100000
loss = np.zeros(epochs)

# Entrenamiento
for epoch in range(epochs):
    linear_model = np.dot(X_train, weights) + bias      # z = W*x+b
    y_pred = sigmoid(linear_model)                      # a = g(z)
    
    # Gradientes
    dw = np.dot(X_train.T, (y_pred - y_train)) / y_train.size
    db = np.sum(y_pred - y_train) / y_train.size
    
    # Actualización
    weights -= learning_rate * dw
    bias -= learning_rate * db
    #print(f"y train {epoch}",y_train)
    #print(f"y pred{epoch}",y_pred)
    # Mostrar pérdida
    loss[epoch] = log_loss(y_train,y_pred)
    if epoch - epochs+1 == 0:
        lossy = log_loss(y_train, y_pred)
        print(f'Época {epoch}, Pérdida: {lossy:.4f}')

def error(loss,i):
    return loss[i-1]-loss[i]

plt.figure(figsize=(8, 4))
eje = np.arange(epochs)
plt.plot(eje,loss)
plt.title(f"Ee")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Magnitud")
plt.grid(True)
plt.show()

# Evaluación en test
y_pred_test = sigmoid(np.dot(X_test, weights) + bias)
y_pred_class_test = [1 if i > 0.5 else 0 for i in y_pred_test]

accuracy = np.mean(y_pred_class_test == y_test)
print(f'Precisión del clasificador en test: {accuracy * 100:.2f}%')
from collections import Counter
print("Predicciones en test:", Counter(y_pred_class_test))
print("Etiquetas reales:", Counter(y_test))

In [ ]:
# Crear una cuadrícula para visualizar los vectores
grid = np.zeros((2, len(y_pred_class_test)))

# Asignar los valores a la cuadrícula
grid[0, :] = y_pred_class_test
grid[1, :] = y_test

# Crear un mapa de colores
cmap = plt.cm.get_cmap('coolwarm', 2)

# Graficar la cuadrícula
plt.imshow(grid, cmap=cmap, aspect='auto')

# Etiquetas y título
plt.title('Comparación de y_pred_class y y')
plt.xlabel('Índice')
plt.ylabel('Vectores')
plt.yticks([0, 1], ['y_pred_class', 'y'])

# Barra de color
cbar = plt.colorbar(ticks=[0, 1])
cbar.ax.set_yticklabels(['0', '1'])

# Mostrar la gráfica
plt.show()

## Softmax

### Entrenamiento

In [ ]:
def entrenamiento_softmax(alpha: np.array, y: np.array , num_classes=3, learning_rate=1.3, max_iterations=10000, tolerance=1e-5):
    """
    Entrena un modelo de regresión logística multinomial usando softmax.
    
    Parámetros:
    - alpha: matriz de características (n_samples x n_features)
    - y: vector de etiquetas (n_samples,)
    - num_classes: número de clases
    - learning_rate: tasa de aprendizaje
    - max_epochs: número máximo de épocas
    - tolerance: tolerancia para la convergencia
    
    Retorna:
    - weights: pesos del modelo entrenado
    - biases: sesgos del modelo entrenado
    """

    import numpy as np
    import matplotlib.pyplot as plt

    # Función softmax
    def softmax(z):
        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)

    # Función de pérdida de entropía cruzada categórica
    def categorical_cross_entropy(y_true, y_pred):
        n = y_true.shape[0]
        epsilon = 1e-15
        y_pred_clipped = np.clip(y_pred, epsilon, 1 - epsilon)
        loss = -np.sum(y_true * np.log(y_pred_clipped)) / n
        return loss

    # Datos de ejemplo (asumiendo que 'alpha' y 'y' están definidos)
    X = np.array(alpha[:])
    y_original = y[:]
    num_classes = 3

    # Convertir las etiquetas a codificación one-hot
    def to_one_hot(y, num_classes):
        one_hot = np.zeros((len(y), num_classes))
        for i, label in enumerate(y):
            one_hot[i, int(label)] = 1
        return one_hot

    y_one_hot = to_one_hot(y_original, num_classes)

    # Dividir los datos manualmente
    np.random.seed(42)
    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)
    split_index = int(0.8 * X.shape[0])
    train_indices = indices[:split_index]
    test_indices = indices[split_index:]

    X_train = X[train_indices]
    y_train_one_hot = y_one_hot[train_indices]
    y_train_original = y_original[train_indices]

    X_test = X[test_indices]
    y_test_one_hot = y_one_hot[test_indices]
    y_test_original = y_original[test_indices]

    # Inicializar pesos y sesgos con valores aleatorios
    num_features = X_train.shape[1]#num_armonicos
    weights = np.random.normal(0, 0.01, (num_features, num_classes)) # Media 0, varianza pequeña (sigma=0.01)
    biases = np.zeros(num_classes)

  
    weights_history_norm = []

    # Entrenamiento del modelo
    relative_diff =2
    print("Entrenando el modelo...")
    previous_weights = weights.copy()
    for epoch in range(max_iterations):
        # Calcular las salidas lineales para el conjunto de entrenamiento
        linear_model_train = np.dot(X_train, weights) + biases
        y_pred_probs_train = softmax(linear_model_train)

        # Calcular el gradiente y actualizar los parámetros
        dw = np.dot(X_train.T, (y_pred_probs_train - y_train_one_hot)) / X_train.shape[0]
        db = np.sum(y_pred_probs_train - y_train_one_hot, axis=0) / X_train.shape[0]
        weights -= learning_rate * dw
        biases -= learning_rate * db

        # Calcular la norma de la matriz de pesos
        weights_norm = np.linalg.norm(weights)

        # Comprobar la condición de parada
        if epoch > 0:
            weight_diff_norm = np.linalg.norm(weights - previous_weights)
            relative_diff = weight_diff_norm / np.linalg.norm(weights) if np.linalg.norm(weights) > 0 else print("ERROR: ",weight_diff_norm)
            if relative_diff < tolerance:
                break
        previous_weights = weights.copy()
        weights_history_norm.append(relative_diff)

    # Evaluación en el conjunto de prueba
    print("\nEvaluación en el conjunto de prueba:")
    linear_model_test = np.dot(X_test, weights) + biases
    y_pred_probs_test = softmax(linear_model_test)
    y_pred_class_test = np.argmax(y_pred_probs_test, axis=1)

    # Calcular la precisión en el conjunto de prueba
    accuracy_test = np.mean(y_pred_class_test == y_test_original)
    print(f'Precisión en el conjunto de prueba: {accuracy_test * 100:.2f}%')

    loss_test = categorical_cross_entropy(y_test_one_hot, y_pred_probs_test)

    return X_test, weights, biases, y_pred_probs_test, y_pred_class_test, y_test_original, accuracy_test

In [ ]:
X_test, weights, biases, y_pred_probs_test, y_pred_class_test, y_test_original, porcentaje_prediccion = entrenamiento_softmax(alpha, y, learning_rate=3)

### Vista

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.cm as cm

num_classes = 3  # Número de clases

n_test_samples = len(y_test_original)

# Definir los colores para cada clase
class_names = ['saxofon', 'clarinete', 'piano']
class_colors = ['orange', 'blue', 'red']
cmap = ListedColormap(class_colors)
bounds = np.arange(num_classes + 1) - 0.5
norm = plt.matplotlib.colors.BoundaryNorm(bounds, cmap.N)

# Crear la figura con dos subplots (2 filas, 1 columna)
fig, axes = plt.subplots(2, 1, figsize=(12, 1), sharex=True)

# Mostrar las predicciones
im1 = axes[0].imshow([y_pred_class_test], cmap=cmap, aspect='auto', norm=norm)
axes[0].set_yticks([0])
axes[0].set_yticklabels(['Predicción'])
#axes[0].set_title('Clases Predichas')

# Mostrar las etiquetas verdaderas
im2 = axes[1].imshow([y_test_original], cmap=cmap, aspect='auto', norm=norm)
axes[1].set_yticks([0])
axes[1].set_yticklabels(['Verdadero'])
axes[1].set_xlabel('                                    ')


plt.show()



## Testeo

In [ ]:
dataset = 'audio_clarinete.csv'
data = pd.read_csv(dataset)


audio_path = data['Path'][0]

In [ ]:
#audio_path = "Clarinete/ClarineteA5(19).wav"
#audio_path = "Piano/PianoA5(18).wav"
audio_path = "Saxo/SaxA5(18).wav"

In [ ]:

signal, rate = cargar_audio(audio_path)
freq, fft_values = calcular_fft(signal, rate)
fft_values = fft_values/fft_values.sum()
alpha_no_norm = np.zeros(len(ticks_frecuencia))
alpha_no_norm = construir_vector_alpha_no_norm(freq,fft_values,ticks_frecuencia,rate)
alpha = construir_vector_alpha(alpha_no_norm)
assert np.abs(np.sum(alpha**2)-1)<1e6

y = etiquetar(data)
X_test = np.array(alpha)
linear_model_test = np.dot(X_test, weights) + biases
y_pred_probs_test = softmax(linear_model_test)
y_pred_class_test = np.argmax(y_pred_probs_test)
if y_pred_class_test == 0:
    print("Predicción: Saxofón")
elif y_pred_class_test == 1:    
    print("Predicción: Clarinete")
elif y_pred_class_test == 2:
    print("Predicción: Piano")

In [ ]:
display(Audio(audio_path,autoplay = True))